In [94]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats 

In [95]:
def load_data():
    std_dev = 1
    mean = 5
    x1 = np.linspace(1, 9, 100)
    x2 = np.linspace(3, 11, 100)
    y = ((x1 > 6) & (x2 > 9)).astype(int)
    X = np.stack((x1, x2), axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], 1))
    return X_train, X_test, y_train, y_test

In [96]:
def initialize_params(X):
    n = X.shape[1]
    w = np.random.randn(n, 1) * 0.1
    b = np.zeros((1, 1))
    params = {'w':w, 'b':b}
    return params

In [97]:
def sigmoid(z):
    a = 1/(1 + np.exp(-z))
    return a

In [98]:
def forward_prop(X, params):
    w = params['w']
    b = params['b']
    z = X @ w + b
    inter_vals = {'z':z}
    yhat = sigmoid(z)
    return yhat, inter_vals

In [99]:
def calculate_cost(yhat, y):
    m = y.shape[0]
    losses = y * np.log(yhat) + (1 - y) * np.log(1 - yhat)
    cost = -np.sum(losses, axis=0, keepdims=True)/m
    return cost

In [123]:
def back_prop(y, yhat, inter_vals, X):
    m = y.shape[0]
    z = inter_vals['z']
    dc_dyhat = (-1/m) * ((y/yhat) - ((1-y)/(1-yhat)))
    
    dyhat_dz = yhat * (1 - yhat)
    dc_dz = dc_dyhat * dyhat_dz
    print(f'dc_dz shape {dc_dz.shape}')
    dc_db = np.sum(dc_dz, axis=0, keepdims=True)
    dz_dw = np.sum(X, axis=0, keepdims=True)
    print(f'dz_dw shape {dz_dw.shape}')
    dc_dw = dc_dz * dz_dw
    print('')
    #print(f'dz_dw {dz_dw}')
    #print(f'dyhat_dz {dyhat_dz}')
    #print(f'dc_dyhat {dc_dyhat}')
    grads = {'dw':dc_dw, 'db':dc_db}
    return grads
    

In [112]:
def update_params(params, grads, lr=0.1):
    w = params['w']
    b = params['b']
    dw = grads['dw']
    db = grads['db']
    print(f'w shape {w.shape}')
    print(f'dw shape {dw.shape}')
    w = w - lr * dw
    b = b - lr * db
    new_parms = {'w':w,'b':b}
    return new_parms

In [113]:
def train_loop(epochs, X_train, y_train, lr):
    params = initialize_params(X_train)
    for epoch in range(epochs):
        yhat, inter_vals = forward_prop(X_train, params)
        print(f'yhat shape {yhat.shape}')
        print(f'z shape {inter_vals['z'].shape}')
        cost = calculate_cost(yhat, y_train)
        grads = back_prop(y_train, yhat, inter_vals, X_train)
        params = update_params(params, grads, lr)
        print(f'epoch {epoch} cost {cost}')

In [114]:
X_train, X_test, y_train, y_test = load_data()

In [115]:
y_train.shape

(80, 1)

In [124]:
epochs = 1000
lr = 0.1

train_loop(epochs, X_train, y_train, lr)

yhat shape (80, 1)
z shape (80, 1)
dc_dz shape (80, 1)
dz_dw shape (1, 2)

w shape (2, 1)
dw shape (80, 2)


ValueError: operands could not be broadcast together with shapes (2,1) (80,2) 

In [66]:
X = np.array([[0.8, 0.9], [0.3, 0.5]])
w = np.array([[0.5], [0.75]])
b = np.array([[1]])
params = {'w':w,'b':b}
yhat, inter_vals = forward_prop(X, params)

[[1.075]
 [0.525]]


In [67]:
yhat

array([[0.88844946],
       [0.82127358]])

In [68]:
y = np.array([[1], [0]])

In [79]:
cost = calculate_cost(yhat, y)

cost vs cost2 [[0.92008826]] : 0.920088258659517


In [70]:
cost

array([[0.92008826]])

In [81]:
grads = back_prop(y, yhat, inter_vals, X)

dz_dw [[1.7]
 [0.8]]
dyhat_dz [[0.09910702]
 [0.14678329]]
dc_dyhat [[-0.56277821]
 [ 2.79757178]]


In [82]:
grads

{'dw': array([[-0.09481796],
        [ 0.32850943]]),
 'db': array([[0.35486152]])}